In [48]:
# model7 inference
# train data: 1800개, eval data: 200개

In [8]:
import torch
from transformers import (
    AutoModelForCausalLM,
    LlamaTokenizerFast,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
    PeftModel
)

In [4]:
base_model = "meta-llama/Llama-2-7b-chat-hf"

In [5]:
# QLoRA 모델을 사용하기 위한 설정

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 
)

In [6]:
# 라마2 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# 토크나이저 로드
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" # 토크나이저 병렬처리 방지(오류 방지)
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true' # __cell__ 오류 방지

tokenizer = LlamaTokenizerFast.from_pretrained(
    base_model,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token # 패딩 토큰을 문장의 끝으로 설정 </s>
tokenizer.padding_side = "right" # 패딩을 문장 뒤에 추가

In [10]:
# 어댑터 추가
model = PeftModel.from_pretrained(model, "/data/ngn_2024/llm_auto_train/models7/checkpoint-112")

In [13]:
# 학습 양식
import json
from datasets import load_dataset
file_name = 'preprocess/dataset3/new_ratings_7.json'

In [14]:
import json

# JSON 파일 불러오기
input_file = file_name
with open(input_file, 'r') as f:
    data = json.load(f)

In [ ]:
# 데이터셋 로드
data = load_dataset('json', data_files=file_name, split="train")

In [15]:
instruction = '''
I will sort the movie list in descending order based on user rating information. I will show the title for the corresponding movie. It must return in JSON format.
'''

In [16]:
# 데이터 매핑 함수 정의
def map_data_to_format(example):

    user_ratings_str = json.dumps(example['ratings'], ensure_ascii=False)
    rank_list_str = json.dumps(example["rank"], ensure_ascii=False)
    
    text = (
        f"###instruction:\n{instruction}\n\n"
        f"user_rating_information:\n{user_ratings_str}\n\n"
        f"###response:\n{rank_list_str}\n\n"
    )
    
    # completion은 모델이 생성해야 할 예상 출력을 포함함
    # 여기서는 우선순위에 따라 정렬된 영화 imdb_id의 리스트를 반환합니다.
    completion = f"{{\"rank\": {rank_list_str}}}"
    
    return {'text': text, 'completion': completion}

In [18]:
# 데이터 매핑 적용
mapped_data = data.map(map_data_to_format)
print(mapped_data)

Dataset({
    features: ['rank', 'userId', 'ratings', 'text', 'completion'],
    num_rows: 2000
})


In [19]:
# 데이터셋 분할
split_data = mapped_data.train_test_split(test_size=0.1)  # 10%를 테스트셋으로 사용

train_set = split_data['train']
eval_set = split_data['test']

train_set = train_set.map(lambda samples: tokenizer(samples["text"], padding=True, truncation=True, return_tensors="pt"), batched=True)
eval_set = eval_set.map(lambda samples: tokenizer(samples["text"], padding=True, truncation=True, return_tensors="pt"), batched=True)


Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [40]:
print(eval_set[90]["text"])

###instruction:

I will sort the movie list in descending order based on user rating information. I will show the title for the corresponding movie. It must return in JSON format.


user_rating_information:
[{"imdbId": "tt120737", "rating": 4.0, "timestamp": 1455356385, "title": "Lord of the Rings: The Fellowship of the Ring, The (2001)"}, {"imdbId": "tt268978", "rating": 4.0, "timestamp": 1455301798, "title": "Beautiful Mind, A (2001)"}, {"imdbId": "tt281358", "rating": 3.0, "timestamp": 1455399329, "title": "Walk to Remember, A (2002)"}, {"imdbId": "tt276751", "rating": 3.5, "timestamp": 1455301898, "title": "About a Boy (2002)"}, {"imdbId": "tt256415", "rating": 3.0, "timestamp": 1455357578, "title": "Sweet Home Alabama (2002)"}, {"imdbId": "tt252076", "rating": 3.0, "timestamp": 1455303078, "title": "Maid in Manhattan (2002)"}, {"imdbId": "tt167261", "rating": 4.0, "timestamp": 1455356588, "title": "Lord of the Rings: The Two Towers, The (2002)"}, {"imdbId": "tt313737", "rating": 3

In [42]:
print(eval_set[90]["ratings"])

[{'imdbId': 'tt120737', 'rating': 4.0, 'timestamp': 1455356385, 'title': 'Lord of the Rings: The Fellowship of the Ring, The (2001)'}, {'imdbId': 'tt268978', 'rating': 4.0, 'timestamp': 1455301798, 'title': 'Beautiful Mind, A (2001)'}, {'imdbId': 'tt281358', 'rating': 3.0, 'timestamp': 1455399329, 'title': 'Walk to Remember, A (2002)'}, {'imdbId': 'tt276751', 'rating': 3.5, 'timestamp': 1455301898, 'title': 'About a Boy (2002)'}, {'imdbId': 'tt256415', 'rating': 3.0, 'timestamp': 1455357578, 'title': 'Sweet Home Alabama (2002)'}, {'imdbId': 'tt252076', 'rating': 3.0, 'timestamp': 1455303078, 'title': 'Maid in Manhattan (2002)'}, {'imdbId': 'tt167261', 'rating': 4.0, 'timestamp': 1455356588, 'title': 'Lord of the Rings: The Two Towers, The (2002)'}, {'imdbId': 'tt313737', 'rating': 3.0, 'timestamp': 1455397999, 'title': 'Two Weeks Notice (2002)'}, {'imdbId': 'tt251127', 'rating': 3.0, 'timestamp': 1455357557, 'title': 'How to Lose a Guy in 10 Days (2003)'}, {'imdbId': 'tt286788', 'ratin

In [41]:
instruction = "I will sort the movie list in descending order based on user rating information. I will show the title for the corresponding movie. It must return in JSON format."

# user_ratings_str3 = eval_set[90]["text"]
# [{"imdbId": "tt101846", "rating": 2.0, "timestamp": 996221356, "title": "F/X2 (a.k.a. F/X 2 - The Deadly Art of Illusion) (1991)"}, {"imdbId": "tt87277", "rating": 3.0, "timestamp": 996259355, "title": "Footloose (1984)"}, {"imdbId": "tt120903", "rating": 2.0, "timestamp": 996212879, "title": "X-Men (2000)"}, {"imdbId": "tt174480", "rating": 2.0, "timestamp": 996217683, "title": "Autumn in New York (2000)"}, {"imdbId": "tt99005", "rating": 5.0, "timestamp": 996256206, "title": "Air America (1990)"}, {"imdbId": "tt191397", "rating": 3.0, "timestamp": 996213058, "title": "Replacements, The (2000)"}, {"imdbId": "tt102510", "rating": 2.0, "timestamp": 996221323, "title": "Naked Gun 2 1/2: The Smell of Fear, The (1991)"}, {"imdbId": "tt204946", "rating": 2.0, "timestamp": 996215785, "title": "Bring It On (2000)"}, {"imdbId": "tt208988", "rating": 4.0, "timestamp": 996217599, "title": "Get Carter (2000)"}, {"imdbId": "tt212338", "rating": 3.0, "timestamp": 996213481, "title": "Meet the Parents (2000)"}]


In [43]:
def generate_output(idx):
    text = f'''
    ###instruction:
    {instruction}


    user_rating_information:
    {eval_set[idx]["ratings"]}


    '''

    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    outputs = model.generate(
        input_ids=inputs["input_ids"].to("cuda"), 
        attention_mask=inputs["attention_mask"], 
        max_new_tokens=256,
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )
    output = tokenizer.decode(outputs[0])
    print(f"--- {idx} 번째 eval set 결과---")
    print(output)



In [44]:
# 사용 예시
for i in range(50):
    generate_output(i) # idx

--- 0 번째 eval set 결과---
<s> 
    ###instruction:
    I will sort the movie list in descending order based on user rating information. I will show the title for the corresponding movie. It must return in JSON format.


    user_rating_information:
    [{'imdbId': 'tt259711', 'rating': 4.0, 'timestamp': 1042841689, 'title': 'Vanilla Sky (2001)'}, {'imdbId': 'tt280778', 'rating': 5.0, 'timestamp': 1042839833, 'title': 'Iris (2001)'}, {'imdbId': 'tt35423', 'rating': 4.0, 'timestamp': 1042841662, 'title': 'Kate & Leopold (2001)'}, {'imdbId': 'tt120737', 'rating': 3.0, 'timestamp': 1042839597, 'title': 'Lord of the Rings: The Fellowship of the Ring, The (2001)'}, {'imdbId': 'tt268995', 'rating': 3.0, 'timestamp': 1042841689, 'title': 'Majestic, The (2001)'}, {'imdbId': 'tt268978', 'rating': 5.0, 'timestamp': 1042841603, 'title': 'Beautiful Mind, A (2001)'}, {'imdbId': 'tt245046', 'rating': 4.0, 'timestamp': 1042841782, 'title': 'Charlotte Gray (2001)'}, {'imdbId': 'tt103129', 'rating': 4.0, 

In [45]:
# 사용 예시2
for i in range(50, 100):
    generate_output(i) # idx

--- 50 번째 eval set 결과---
<s> 
    ###instruction:
    I will sort the movie list in descending order based on user rating information. I will show the title for the corresponding movie. It must return in JSON format.


    user_rating_information:
    [{'imdbId': 'tt108598', 'rating': 4.0, 'timestamp': 965705539, 'title': 'Wallace & Gromit: The Wrong Trousers (1993)'}, {'imdbId': 'tt99703', 'rating': 2.0, 'timestamp': 965712263, 'title': 'Grifters, The (1990)'}, {'imdbId': 'tt73486', 'rating': 4.0, 'timestamp': 965702660, 'title': "One Flew Over the Cuckoo's Nest (1975)"}, {'imdbId': 'tt80684', 'rating': 4.0, 'timestamp': 965703555, 'title': 'Star Wars: Episode V - The Empire Strikes Back (1980)'}, {'imdbId': 'tt93779', 'rating': 5.0, 'timestamp': 965705725, 'title': 'Princess Bride, The (1987)'}, {'imdbId': 'tt82971', 'rating': 5.0, 'timestamp': 965709394, 'title': 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)'}, {'imdbId': 'tt90605', 'rating': 3.0, '

In [46]:
# 사용 예시3
for i in range(100, 150):
    generate_output(i) # idx

--- 100 번째 eval set 결과---
<s> 
    ###instruction:
    I will sort the movie list in descending order based on user rating information. I will show the title for the corresponding movie. It must return in JSON format.


    user_rating_information:
    [{'imdbId': 'tt111282', 'rating': 3.0, 'timestamp': 830939438, 'title': 'Stargate (1994)'}, {'imdbId': 'tt114924', 'rating': 5.0, 'timestamp': 830939488, 'title': 'While You Were Sleeping (1995)'}, {'imdbId': 'tt110598', 'rating': 4.0, 'timestamp': 830939941, 'title': "Muriel's Wedding (1994)"}, {'imdbId': 'tt99348', 'rating': 5.0, 'timestamp': 830940157, 'title': 'Dances with Wolves (1990)'}, {'imdbId': 'tt96895', 'rating': 4.0, 'timestamp': 830939391, 'title': 'Batman (1989)'}, {'imdbId': 'tt101414', 'rating': 3.0, 'timestamp': 830939438, 'title': 'Beauty and the Beast (1991)'}, {'imdbId': 'tt113627', 'rating': 2.5, 'timestamp': 1100803583, 'title': 'Leaving Las Vegas (1995)'}, {'imdbId': 'tt111255', 'rating': 0.5, 'timestamp': 1100803

In [47]:
# 사용 예시4
for i in range(150, 200):
    generate_output(i) # idx

--- 150 번째 eval set 결과---
<s> 
    ###instruction:
    I will sort the movie list in descending order based on user rating information. I will show the title for the corresponding movie. It must return in JSON format.


    user_rating_information:
    [{'imdbId': 'tt68503', 'rating': 4.5, 'timestamp': 1520409485, 'title': 'Dr. Phibes Rises Again (1972)'}, {'imdbId': 'tt79489', 'rating': 4.0, 'timestamp': 1520409551, 'title': 'Love at First Bite (1979)'}, {'imdbId': 'tt96386', 'rating': 4.0, 'timestamp': 1520409558, 'title': 'Life Is a Long Quiet River (La vie est un long fleuve tranquille) (1988)'}, {'imdbId': 'tt318403', 'rating': 4.5, 'timestamp': 1520408735, 'title': 'Lion King 1½, The (2004)'}, {'imdbId': 'tt374277', 'rating': 4.0, 'timestamp': 1520409031, 'title': 'Touch of Pink (2004)'}, {'imdbId': 'tt367594', 'rating': 2.0, 'timestamp': 1520408421, 'title': 'Charlie and the Chocolate Factory (2005)'}, {'imdbId': 'tt386588', 'rating': 1.5, 'timestamp': 1520408378, 'title': 'Hitc

In [34]:
# # {instruction}

# text = f'''
# ###instruction:
# {instruction}


# user_rating_information:
# {eval_set[90]["text"]}


# '''
# inputs = tokenizer(text, return_tensors="pt").to("cuda")
# outputs = model.generate(
#     input_ids=inputs["input_ids"].to("cuda"), 
#     attention_mask=inputs["attention_mask"], 
#     max_new_tokens=256,
#     early_stopping=True,
#     pad_token_id=tokenizer.eos_token_id
# )
# output = tokenizer.decode(outputs[0])
# print(output)